In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger,ReduceLROnPlateau

In [5]:
%load_ext tensorboard

In [6]:
import os
from pathlib import Path
import matplotlib.pylab as plb
import numpy as np
import datetime, math
import pandas as pd


In [7]:
tf.get_logger().setLevel('INFO')

In [8]:
splits, info = tfds.load('horses_or_humans', as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])


Shuffling and writing examples to /home/hasan/tensorflow_datasets/horses_or_humans/3.0.0.incomplete2FJ2K9/horses_or_humans-train.tfrecord


Shuffling and writing examples to /home/hasan/tensorflow_datasets/horses_or_humans/3.0.0.incomplete2FJ2K9/horses_or_humans-test.tfrecord


Dataset horses_or_humans downloaded and prepared to /home/hasan/tensorflow_datasets/horses_or_humans/3.0.0. Subsequent calls will reuse this data.


In [9]:
(train_examples, validation_examples, test_examples) = splits

In [10]:
num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [11]:
SIZE = 150 
IMAGE_SIZE = (SIZE, SIZE)

In [12]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [13]:
BATCH_SIZE = 32

In [14]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [15]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [16]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

In [17]:
!rm -rf logs#don't know why

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])

In [23]:

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S"))

In [24]:
logdir

'logs/2020_12_06-19_51_39'

In [25]:
tensorboard= TensorBoard(logdir)

In [26]:
tensorboard

In [30]:
#os.environ['TENSORBOARD_BINARY'] = '/home/hasan/anaconda3/envs/tf'

In [31]:
model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard])

Epoch 1/10
26/26 [==============================] - 3s 113ms/step - loss: 0.1032 - accuracy: 0.9805 - val_loss: 0.0764 - val_accuracy: 0.9902
Epoch 2/10
26/26 [==============================] - 3s 112ms/step - loss: 0.0824 - accuracy: 0.9818 - val_loss: 0.0927 - val_accuracy: 0.9659
Epoch 3/10
26/26 [==============================] - 3s 114ms/step - loss: 0.0720 - accuracy: 0.9854 - val_loss: 0.0682 - val_accuracy: 0.9805
Epoch 4/10
26/26 [==============================] - 3s 111ms/step - loss: 0.0689 - accuracy: 0.9805 - val_loss: 0.0542 - val_accuracy: 0.9854
Epoch 5/10
26/26 [==============================] - 3s 111ms/step - loss: 0.0573 - accuracy: 0.9878 - val_loss: 0.0514 - val_accuracy: 0.9805
Epoch 6/10
26/26 [==============================] - 3s 111ms/step - loss: 0.0506 - accuracy: 0.9903 - val_loss: 0.0457 - val_accuracy: 0.9805
Epoch 7/10
26/26 [==============================] - 3s 111ms/step - loss: 0.0485 - accuracy: 0.9891 - val_loss: 0.0714 - val_accuracy: 0.9659
Epoch 

In [32]:
%tensorboard --logdir logs

ERROR: Failed to start '/home/hasan/anaconda3/envs/tf' (set by the
`TENSORBOARD_BINARY` environment variable): [Errno 13] Permission
denied: '/home/hasan/anaconda3/envs/tf'

In [33]:
Path.cwd()

PosixPath('/home/hasan/Schreibtisch/projects/coursera/tensorflow')

 [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

In [66]:
checkpoint_path = Path('tmp/checkpoint')

In [67]:
checkpoint_path

PosixPath('tmp/checkpoint')

In [68]:
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])

EPOCHS = 10
checkpoint_filepath = checkpoint_path
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [69]:
checkpoint_path

PosixPath('tmp/checkpoint')

In [70]:
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[model_checkpoint_callback])

Epoch 1/5
26/26 - 3s - loss: 0.0144 - accuracy: 0.9964 - val_loss: 0.0237 - val_accuracy: 0.9902
Epoch 2/5
26/26 - 3s - loss: 0.0133 - accuracy: 0.9988 - val_loss: 0.0198 - val_accuracy: 0.9902
Epoch 3/5
26/26 - 3s - loss: 0.0142 - accuracy: 0.9988 - val_loss: 0.0234 - val_accuracy: 0.9902
Epoch 4/5
26/26 - 3s - loss: 0.0113 - accuracy: 0.9988 - val_loss: 0.0187 - val_accuracy: 0.9951
Epoch 5/5
26/26 - 3s - loss: 0.0111 - accuracy: 0.9988 - val_loss: 0.0175 - val_accuracy: 0.9902


In [71]:
#model.load_weights(checkpoint_filepath)

In [74]:
chckpoint = ModelCheckpoint('tmp/weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1, save_best_only=True,monitor='val_accuracy',
    mode='max')

In [75]:
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[chckpoint])

Epoch 1/5

Epoch 00001: val_accuracy improved from -inf to 0.99512, saving model to tmp/weights.01-0.02.h5
26/26 - 3s - loss: 0.0070 - accuracy: 1.0000 - val_loss: 0.0168 - val_accuracy: 0.9951
Epoch 2/5

Epoch 00002: val_accuracy did not improve from 0.99512
26/26 - 3s - loss: 0.0065 - accuracy: 1.0000 - val_loss: 0.0155 - val_accuracy: 0.9951
Epoch 3/5

Epoch 00003: val_accuracy did not improve from 0.99512
26/26 - 3s - loss: 0.0058 - accuracy: 1.0000 - val_loss: 0.0160 - val_accuracy: 0.9951
Epoch 4/5

Epoch 00004: val_accuracy did not improve from 0.99512
26/26 - 3s - loss: 0.0060 - accuracy: 1.0000 - val_loss: 0.0153 - val_accuracy: 0.9951
Epoch 5/5

Epoch 00005: val_accuracy did not improve from 0.99512
26/26 - 3s - loss: 0.0052 - accuracy: 1.0000 - val_loss: 0.0143 - val_accuracy: 0.9951


In [76]:
checkPoint= ModelCheckpoint('saved_model', verbose=1)

In [77]:
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[checkPoint])

Epoch 1/5

Epoch 00001: saving model to saved_model
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: saved_model/assets


...

26/26 - 3s - loss: 0.0046 - accuracy: 1.0000 - val_loss: 0.0147 - val_accuracy: 0.9951
Epoch 4/5

Epoch 00004: saving model to saved_model
INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 3s - loss: 0.0043 - accuracy: 1.0000 - val_loss: 0.0162 - val_accuracy: 0.9951
Epoch 5/5

Epoch 00005: saving model to saved_model
INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 3s - loss: 0.0040 - accuracy: 1.0000 - val_loss: 0.0142 - val_accuracy: 0.9951


In [78]:
chkpoint= ModelCheckpoint('model.h5', verbose=1)

In [79]:
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[chkpoint])

Epoch 1/5

Epoch 00001: saving model to model.h5
26/26 - 3s - loss: 0.0039 - accuracy: 1.0000 - val_loss: 0.0132 - val_accuracy: 0.9951
Epoch 2/5

Epoch 00002: saving model to model.h5
26/26 - 3s - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.0141 - val_accuracy: 0.9951
Epoch 3/5

Epoch 00003: saving model to model.h5
26/26 - 3s - loss: 0.0035 - accuracy: 1.0000 - val_loss: 0.0144 - val_accuracy: 0.9951
Epoch 4/5

Epoch 00004: saving model to model.h5
26/26 - 3s - loss: 0.0034 - accuracy: 1.0000 - val_loss: 0.0151 - val_accuracy: 0.9951
Epoch 5/5

Epoch 00005: saving model to model.h5
26/26 - 3s - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0.0129 - val_accuracy: 0.9951


 [Early stopping](https://keras.io/api/callbacks/early_stopping/)

In [80]:
early_stop = EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)

In [81]:
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[early_stop])

Epoch 1/5
26/26 - 3s - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.0226 - val_accuracy: 0.9951
Epoch 2/5
26/26 - 3s - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.0153 - val_accuracy: 0.9951
Epoch 3/5
26/26 - 3s - loss: 0.0028 - accuracy: 1.0000 - val_loss: 0.0130 - val_accuracy: 0.9951
Epoch 4/5
Restoring model weights from the end of the best epoch.
26/26 - 3s - loss: 0.0028 - accuracy: 1.0000 - val_loss: 0.0147 - val_accuracy: 0.9951
Epoch 00004: early stopping


 [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

In [82]:
csv_file = CSVLogger('training.csv')
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[csv_file])

Epoch 1/5
26/26 - 3s - loss: 0.0034 - accuracy: 1.0000 - val_loss: 0.0133 - val_accuracy: 0.9951
Epoch 2/5
26/26 - 3s - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.0148 - val_accuracy: 0.9951
Epoch 3/5
26/26 - 3s - loss: 0.0028 - accuracy: 1.0000 - val_loss: 0.0136 - val_accuracy: 0.9951
Epoch 4/5
26/26 - 3s - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0152 - val_accuracy: 0.9951
Epoch 5/5
26/26 - 3s - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.0139 - val_accuracy: 0.9951


[Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

In [83]:
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 3s 115ms/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0143 - val_accuracy: 0.9951

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 3s 111ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0141 - val_accuracy: 0.9951

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 3s 113ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0141 - val_accuracy: 0.9951

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 3s 114ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0140 - val_accuracy: 0.9951

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 3s 113ms/step - loss: 0.0023 -

In [84]:
%tensorboard --logdir log_dir

ERROR: Failed to start '/home/hasan/anaconda3/envs/tf' (set by the
`TENSORBOARD_BINARY` environment variable): [Errno 13] Permission
denied: '/home/hasan/anaconda3/envs/tf'

[ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)
Reduce learning rate when a metric has stopped improving.

In [ ]:
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 3s 113ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0140 - val_accuracy: 0.9951
Epoch 2/50
26/26 [==============================] - 3s 112ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0140 - val_accuracy: 0.9951
Epoch 3/50
26/26 [==============================] - 3s 111ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0140 - val_accuracy: 0.9951
Epoch 4/50
26/26 [==============================] - 3s 114ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0140 - val_accuracy: 0.9951
Epoch 5/50
26/26 [==============================] - 3s 111ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0140 - val_accuracy: 0.9951
Epoch 6/50
26/26 [==============================] - 3s 112ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0140 - val_accuracy: 0.9951
Epoch 7/50
26/26 [==============================] - 3s 113ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0141 - val_accuracy: 0.9951
Epoch 